# AM207 Final Project: Latent Dirichlet Allocation

### Cole Diamond
### Wei Dai
### Raphael Pestourie

1. Finding a list of documents that we want to study (LDA is using a 'bag of words model', there should be an app that transform PDF into bags of words on the internet)
2. Preprocess:
    a. get a probability of each character and suppress the words with high probability from our dictionary (assign each word to one index)
    b. create a random Z array that matches words to a topic number and find the corresponding 'n' 3D matrix
3. Sampling with Collapsed Gibbs sampling to get the Z matrix from our data and find the corresponding 'n' 3D matrix
4. Recover hidden variables from the Z matrix.

---

##1) Get text data from a list of documents and transform into a bag of words

I chose the publications of my former professor Prof. Zhang at UC Berkeley.

In [1]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO

def convert_pdf_to_txt(path):
    """
    This function converts a pdf document from my computer into a str
    path: path to the pdf I want to convert
    str: utf8 text extracted from the pdf
    """
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    fp.close()
    device.close()
    str = retstr.getvalue()
    retstr.close()
    return str

In [2]:
"""
This function transform the pdf into strings of text
"""

import glob, os
import numpy as np

# Compute the number of documents in the directory
os.chdir("/home/raphael/xlab.me.berkeley.edu/pdf")
i=0
for file in glob.glob("*.pdf"):
    i+=1

# create a list of list, each list will be the text file of one pdf
documents = [[] for k in range(i)]

#fill in the documents list of list
for j, file in enumerate(glob.glob("*.pdf")):        
    fname="/home/raphael/xlab.me.berkeley.edu/pdf/"+str(file)
    documents[j] = convert_pdf_to_txt(fname)

In [3]:
"""
function to load .txt files and use it as is
"""

'\nfunction to load .txt files and use it as is\n'

In [4]:
# get a bag of words (all lower case)
import re

# clean list
documents_wordlist = [[] for k in range(len(documents))]

for j, mystr in enumerate(documents):
    # take only the words
    documents_wordlist[j] = re.sub("[^\w]", " ",  mystr).split()
    # lowercase them
    for k in range(len(documents_wordlist[j])):
        documents_wordlist[j][k] = documents_wordlist[j][k].lower()

In [5]:
"""
function to save the .txt file
"""
f = open("documents_wordlist.txt","a")
for doc in documents_wordlist:
    for word in doc:
        f.write("%s " % word)
    f.write("\n")

##2) a) Counting words, create a dictionary and remove the words that are too common or too rare

In [6]:
"""
This function creates a set out of the words of 'documents_wordlist'
input: documents_wordlist
output: set of all the words
"""

# create a set with all the words
concat = []

#concatenate all the word list
for i in range(0, len(documents_wordlist)):
    concat+= documents_wordlist[i]

#create a set
set_words = set(concat)

In [7]:
"""
This function reduces the set of words we are interested in.
It suppress all the words that are to common accross the documents, or too rare

input
sup: probability above which words are removed
inf: probability below which words are removed
documents_wordlist: list of document's baggs of words

output
final_set_words: set of the words we are interested in
"""

# create a dictionary for the probabilities of each word 
# which permits to suppress less common words, or too common ones)

#initialization
dictionary_process = {word: [] for word in set_words}

# create a dictionary where we append the index of the document to the value of the word
# we don't append it twice
for i in range(len(documents_wordlist)):
    for word in documents_wordlist[i]:
        if i not in dictionary_process[word]:
            dictionary_process[word].append(i)

final_set_words = set(set_words)
# remove the words that appears in less that 5% of the documents of in 
# more than 70% (not to take 'Etcheverry' into account)
# or more than 80% (to still take 'metal' into account)
for word in set_words:
    if len(dictionary_process[word])/float(len(documents_wordlist))>.8:
        final_set_words.remove(word)
    else:
        if len(dictionary_process[word])/float(len(documents_wordlist))<.07:
            final_set_words.remove(word)    

In [8]:
print (len(set_words)-len(final_set_words))/float(len(set_words)), " decrease in the number of words"

0.911651764442  decrease in the number of words


In [9]:
# we  wand the number of a word to be always the same
global sorted_list
sorted_list = sorted(final_set_words)

In [10]:
def create_countdict_per_doc(document):
    """
    This function takes a document and the global sorted_list 
    and creates a dictionnay with all the words in sorted_list with their numbers of occurences in the document
    input: document
    output: dictionarry
    """
    dictionary_per_doc = {word: 0 for word in sorted_list}
    for word in document:
        if word in dictionary_per_doc.keys():
            dictionary_per_doc[word]+= 1
    return dictionary_per_doc

In [21]:
def create_dict_list(documents_wl):
    """
    This function creates a counting dictionnary for each documents in the document list
    This is useful each time we compute the 3D n matrix
    input: list of documents
    output: list of dictionaries
    """
    dict_list = []
    for doc in documents_wl:
        dict_list.append(create_countdict_per_doc(doc))
    
    return dict_list

In [22]:
global dict_list
dict_list = create_dict_list(documents_wordlist)

In [23]:
# Save a dictionary into a pickle file.
import pickle

pickle.dump( dict_list, open( "save_dict_list.p", "wb" ) )

##2) b) Initialization of Z and function to compute n from Z

In [72]:
global D, V
D = len(documents_wordlist)
V = len(sorted_list)

In [41]:
# number of topic we want
K = 7

In [42]:
Z = np.random.randint(0,K-1, len(sorted_list))

In [43]:
def n_3Dmatrix(K, Z):
    """
    This function creates the 3D 'n' matrix that is needed for the collapsed gibbs sampling
    one dimension is the topics (this dimension is a list),
    the 2 other dimensions is a matrix with card(Z==k) (which varies) columns and D rows
    Z: the array that links words to topics
    K: number of topic
    """
    
    n = []
    for k in range(K):
        # find the words related to this topic
        indexes =  np.where(Z == k)[0]
        
        # initialize the matrix
        count_mat = np.zeros((D, len(indexes)))
        
        # for each word (that we get thanks to sorted_list) 
        # go through the documents dictionaries and take the counts
        for i, index in enumerate(indexes):
            for j in range(D):
                count_mat[j, i] = dict_list[j][sorted_list[index]]
        
        # add the matrix to the list of matrices
        n.append(count_mat)
    
    return n

##3) Sampling with Collapsed Gibbs sampling to get the Z matrix from our data and find the corresponding 'n' 3D matrix

From here we need:

- dict_list
- sorted_list
- documents_wordlist (for Gibbs sampler)

$p(Z_{(m,n)}=k|Z_{-(m,n)},W,\alpha,\beta)=\left(n_{m,(\cdot)}^{k,-(m,n)}+\alpha_{k}\right)\frac{n_{(\cdot),v}^{k,-(m,n)}+\beta_{v}}{\sum_{r=1}^{V}n_{(\cdot),r}^{k,-(m,n)}+\beta_{r}}
 $

In [82]:
def joint_probability_fn(m, n, Z, alpha, beta):
    """
    Compute the value of the joint probability using K and Z given the data
    Return term 1 * term 2 / term 3
    
    input
    m: doc nb we are on
    n: word nb we are on (index in the sorted list) we need an enumerate to find it in the matrix
    k: sampled number of topic assigned to (m,n) which count can be zero
    Z: 
    alpha
    beta
    
    output
    result: value of the star joint probability
    """
    n_mat = n_3Dmatrix(K, Z)
    k = Z[n]
    
    # indexes of the word in absolute value
    indexes =  np.where(Z == k)[0]
    
    #find index of n in the 'n_mat' 3d matrix
    for i, index in enumerate(indexes):
        if index==n:
            index_searched = i
        
    # value to substract (this is my understanding of the -(m, n))
    substract_val = n_mat[k][m, index_searched]
    
    # 1st term
    term1 = np.sum(n_mat[k][m,:]) - substract_val + alpha[k]
    
    # 2nd term
    # note how tricky it is here, the index in n_mat and the index in beta (that of the sorted_list)
    # are not the same !!! Even though of course it refers to the same word.
    term2 = np.sum(n_mat[k][:, index_searched]) - substract_val + beta[n]
    
    # 3rd term
    # POTENTIAL ERROR : here I read r, as the word in the columns of the topic matrix
    # go back and see what V is
    term3 = np.sum([np.sum(n_mat[k][:, i]) - substract_val + beta[index] for i, index in enumerate(indexes)])
    
    return (term1 * term2 / float(term3))

##?

alpha and beta are hyperparameter?

They never change? (make sure I understood) -> hierarchical Bayes?

In [110]:
# initialize alpha
alpha = np.ones(K)

#initialize beta
beta = np.ones(V)

In [119]:
def iteration_gibbs_WRONG(Z, alpha, beta):
    """
    This function does one iteration of Gibbs sampling
    input: 
    Z: assignment of each word to a topic
    """
    
    trace_z1000 = np.zeros(K*D)
    
    count = 0

    #train only on 1 tenth of the articles
    for m in range(0,D,10):
        # ATTENTION! this one might be an issue because the n matrix changes when Z changes
        for l in range(K):
            trace_z1000[count] = Z[1000] 
            count+= 1
            print Z[1000] 
            indexes =  np.where(Z == l)[0]
            for n in indexes:
                Z_star = np.copy(Z)
                new_k = np.random.randint(0,K-1)
                Z_star[n] = new_k
                p_current = joint_probability_fn(m, n, Z, alpha, beta)
                p_star = joint_probability_fn(m, n, Z_star, alpha, beta)
                ratio = p_star / p_current
                if np.random.rand() < ratio:
                    Z = np.copy(Z_star)

                    
    return Z, trace_z1000

In [150]:
def iteration_gibbs(Z, alpha, beta):
    """
    This function does one iteration of Gibbs sampling
    input: 
    Z: assignment of each word to a topic
    """
    
    trace = np.zeros((len(sorted_list), D))
    

#     go trhough all the word in all the articles
    for j, doc in enumerate(documents_wordlist):
        trace[:, j] = Z
        for word in doc:
            for i, word_check in enumerate(sorted_list):
#                 check if the word are in the sorted_list
                if word_check==word:
#                     find the index i
                    n=i
                    
#                     sample a new topic for that word
                    Z_star = np.copy(Z)
                    new_k = np.random.randint(0,K-1)
                    Z_star[n] = new_k
#                     calculate joint probabilities
                    p_current = joint_probability_fn(j, n, Z, alpha, beta)
                    p_star = joint_probability_fn(j, n, Z_star, alpha, beta)
                    ratio = p_star / p_current
                    if np.random.rand() < ratio:
                        Z = np.copy(Z_star)

                    
    return Z, trace

In [ ]:
#### Gibbs sampling ####

# nb of iteration
N_it = 1

# tracer for the Z[1000]s, only one value because the whole array is too costly, 1000 is arbitrary
tracer_one_value = []

#initializaiton of the Z matrix (the only variables we are sampling!!! because it is collapsed Gibbs sampling)
Z = np.random.randint(0,K-1, len(sorted_list))

for iteration in range(N_it):
    Z, tracer_toappend = iteration_gibbs(Z, alpha, beta)
    tracer_one_value.append(tracer_toappend)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Pray and tmr you'll see it converged
plt.plot(tracer_one_value[50, :])